In [1]:
#!pip unistall -y transformers
!pip install -U -q "transformers==4.38.0" --upgrade

In [2]:
### Get secret and login to huggingface
# from google.colab import userdata
# from huggingface_hub import login
# login(userdata.get('hf'))


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
## Load Gemma from huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Prepare prompt template for fake statement detection


In [13]:

def get_completion_gemma(statement,context):

    prompt = f"""Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

    Statement:
    {statement}

    Context:
    {context}

    Response:
    """

    #print(prompt)
    input_ids = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**input_ids, max_time=20, max_new_tokens=100, temperature=0.8)
    return tokenizer.decode(outputs[0])


### Prepare the following testcases to test the above prompt and the responses from Gemma
* Will run the same prompt few times to get the probability of getting consistent results
* If run without GPU, it reserves about 18G ram and consumes ~11G ram for running; barely works normal CPU on colab, but easy to get nil response if max_time is short, say 10s


### Case 1: A fake statement with authorized context found online
* Expect to flag fake


In [14]:
if False:
    statement = 'Joe Biden said not vote.'
    context = '''
    Fake Joe Biden robocall tells New Hampshire Democrats not to vote Tuesday
    The call, an apparent imitation or digital manipulation of the president's voice, says, "Voting this Tuesday only enables the Republicans in their quest to elect Donald Trump again."
    
    MANCHESTER, N.H. — The New Hampshire attorney general's office says it is investigating what appears to be an "unlawful attempt" at voter suppression after NBC News reported on a robocall impersonating President Joe Biden that told recipients not to vote in Tuesday's presidential primary.
    '''
    
    for ii in range(0,3):
      print(f'Trial {ii}')
      output = get_completion_gemma(statement, context)
      print(output)


Trial 0
<bos>Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

    Statement:
    Joe Biden said not vote.

    Context:
    
    Fake Joe Biden robocall tells New Hampshire Democrats not to vote Tuesday
    The call, an apparent imitation or digital manipulation of the president's voice, says, "Voting this Tuesday only enables the Republicans in their quest to elect Donald Trump again."
    
    MANCHESTER, N.H. — The New Hampshire attorney general's office says it is investigating what appears to be an "unlawful attempt" at voter suppression after NBC News reported on a robocall impersonating President Joe Biden that told recipients not to vote in Tuesday's presidential primary.
    

    Response:
    The statement is fake.
    
    
Trial 1
<bos>Determine if the given statement is fake, based on the context below.
  

### Case 2: A simple statement with the same content in context
  * Expect to flag true or I dont know


In [15]:
if False:
    statement = 'This is good.'
    context = 'This is good.'
    
    for ii in range(0,3):
      print(ii)
      output = get_completion_gemma(statement, context)
      print(output)


0
<bos>Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

    Statement:
    This is good.

    Context:
    This is good.

    Response:
    I don't know.

    Answer:
1
<bos>Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

    Statement:
    This is good.

    Context:
    This is good.

    Response:
    I don't know.

    Answer:

2
<bos>Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

    Statement:
    This is good.

    Context:
    This is good.

    Response:
    I don't know.

   

### Case 3: A true statement with the same content in context
  * Expect to flag true

In [16]:
if False:
    statement = 'Joe Biden is the current US President.'
    context = statement
    
    
    for ii in range(0,3):
      print(ii)
      output = get_completion_gemma(statement, context)
      print(output)


0
<bos>Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

    Statement:
    Joe Biden is the current US President.

    Context:
    Joe Biden is the current US President.

    Response:
    I don't know.

    Answer:

1
<bos>Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

    Statement:
    Joe Biden is the current US President.

    Context:
    Joe Biden is the current US President.

    Response:
    I don't know.

    Answer:

2
<bos>Determine if the given statement is fake, based on the context below.
    If you don't know, say that you do not know.

    Use only information from the context to make the decision.  Answer the question at the end.

 